In [1]:
#!pip install kafka-python

In [2]:
#!pip install tqdm

In [3]:
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
from json import dumps
from tqdm import tqdm

In [4]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
if producer.bootstrap_connected():
    print(f"Successfully connected to bootstrap server")
else:
    print("Couldn't connect to bootstrap server.")

TOPIC_NAME = "ml-raw-dns"

Successfully connected to bootstrap server


In [5]:
def produce_message(producer_instance, topic, message):
    producer_instance.send(topic, message)
    producer_instance.flush()
    return

In [6]:
# Load
df = pd.read_csv("combined_test_df.csv")

# Drop auto‑generated index column if present
df.drop(columns=["Unnamed: 0"], inplace=True, errors="ignore")

# Convert labels to binary
df["label"] = df["label"].apply(lambda x: 0 if x == "BENIGN" else 1)

# Specify the top features exactly
best_features = [
    "bwd packets/s",
    "destination port",
    "init_win_bytes_forward",
    "flow packets/s",
    "bwd packet length min",
    "down/up ratio",
    "psh flag count",
    "act_data_pkt_fwd",
    "total fwd packets",
    "subflow fwd bytes",
]

# Select only those columns + the label
df = df[best_features + ["label"]]

# Shuffle and reset index
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save
df.to_csv("combined_test_df_phase2.csv", index=False)

print("Dataset updated: dropped 'Unnamed: 0', converted labels, reshuffled rows.")
print("New shape:", df.shape)
print(df.head())

Dataset updated: dropped 'Unnamed: 0', converted labels, reshuffled rows.
New shape: (53576, 11)
   bwd packets/s  destination port  init_win_bytes_forward  flow packets/s  \
0      44.725720                53                      -1       89.451439   
1       0.000000                80                     256        0.497906   
2       2.874044             59641                     229        3.353051   
3       0.436495               443                   29200        0.872991   
4       0.000000                50                   29231     -397.150515   

   bwd packet length min  down/up ratio  psh flag count  act_data_pkt_fwd  \
0                     77              1               0                 1   
1                      0              0               0                 3   
2                      6              6               0                 0   
3                      0              1               1                 6   
4                      1              0          

In [7]:
with open("combined_test_df_phase2.csv") as f:
    start_time = datetime.now()
    for i, line in tqdm(enumerate(f)):
        produce_message(producer_instance=producer, topic=TOPIC_NAME, message=line)
    end_time = datetime.now()
    print(f"Batch took {end_time-start_time} time for ingesting data")

print("Ingestion Completed")

53577it [06:52, 129.94it/s]

Batch took 0:06:52.456893 time for ingesting data
Ingestion Completed
